## MICCAI Test Pipeline (bilinear variants)

### Imports

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from imports import *

In [10]:
# overall imports
import importlib
import data

# importlib.reload(data.data_utils)
# importlib.reload(data.data_load)
#importlib.reload(sim.sim_utils)
# importlib.reload(models)

import models
import sim.sim 
import sim.sim_utils
from sim.sim_utils import bytes2human, print_system_usage
from sim.sim import Simulation
from sim.sim_run import single_sim_run, open_pickled_results


#### Check job specs

In [11]:
print_system_usage()

total = psutil.disk_usage('/').total
print(bytes2human(total))

CPU Usage: 18.8%
RAM Usage: 9.2%
Available RAM: 342.4G
Total RAM: 377.1G
52.4G


In [12]:
print("XGBoost version:", xgboost.__version__)
print("cupy version:", cp.__version__)

XGBoost version: 2.0.3
cupy version: 13.1.0


In [13]:
GPUtil.getAvailable()
# if a number is seen a GPU is available

[]

In [14]:
GPUtil.getGPUs()

DEVICE_ID_LIST = GPUtil.getFirstAvailable()
DEVICE_ID = DEVICE_ID_LIST[0] # grab first element from list
if DEVICE_ID != None: 
    print('GPU found', DEVICE_ID)
    use_gpu = True

RuntimeError: Could not find an available GPU after 1 attempts with 900 seconds interval.

In [19]:
GPUtil.showUtilization()


| ID | GPU | MEM |
------------------
|  0 |  0% |  2% |


In [20]:
torch.cuda.is_available()

# Clear GPU memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [10]:
torch.cuda.empty_cache()

## Start test pipeline <a id="sims"></a>

In [ ]:
def run_simulation_set(model_types=['dynamic_mlp'],
                      cv_types=['spatial', 'random'],
                      parcellations=['S100', 'S400'],
                      connectome_targets=['FC', 'SC'],
                      feature_types=['transcriptome', 'euclidean'],
                      hemisphere='left',
                      random_seeds=[42, 123, 456]):
    """
    Run a set of simulations with different combinations of parameters.
    
    Args:
        model_types (list): List of model types to test
        cv_types (list): List of cross-validation types
        parcellations (list): List of parcellation schemes
        connectome_targets (list): List of connectome targets
        feature_types (list): List of feature types
        hemisphere (str): Brain hemisphere to use ('left', 'right', 'both')
        random_seeds (list): List of random seeds for multiple runs
    """
    
    for seed in random_seeds:
        for model in model_types:
            for cv in cv_types:
                for parc in parcellations:
                    # Set hemisphere based on parcellation
                    current_hemisphere = 'both' if parc == 'S100' else 'left'
                    
                    for target in connectome_targets:
                        for feat in feature_types:
                            # Configure feature type dict
                            feat_dict = [{'transcriptome': None}] if feat == 'transcriptome' else [{'euclidean': None}]
                            
                            print(f"Running simulation with: {model}, {cv}, {parc}, {target}, {feat}, seed={seed}")
                            
                            # Run single simulation
                            single_sim_run(
                                cv_type=cv,
                                random_seed=seed,
                                model_type=model,
                                feature_type=feat_dict,
                                connectome_target=target,
                                use_gpu=True,
                                use_shared_regions=False,
                                test_shared_regions=False,
                                omit_subcortical=False,
                                parcellation=parc,
                                gene_list='0.2',
                                hemisphere=current_hemisphere,
                                search_method=('wandb', 'mse', 10),
                                save_sim=False,
                                track_wandb=True,
                                skip_cv=False
                            )
                            
                            # Clear GPU memory
                            if torch.cuda.is_available():
                                torch.cuda.empty_cache()
                            # Clear CPU memory
                            gc.collect()

# Example usage:
run_simulation_set(
    model_types=['bilinear_lowrank', 'shared_mlp_encoder', 'shared_linear_encoder'],
    cv_types=['random', 'spatial'],
    parcellations=['S100', 'S400'],
    connectome_targets=['FC', 'SC'],
    feature_types=['transcriptome'],
    hemisphere='both',
    random_seeds=[1, 2]
)